# 1. Import

In [ ]:
!pip install Sastrawi
!pip install ftfy
!pip install xgboost
!pip install openpyxl
!pip install googletrans
!pip install tqdm
!pip install concurrent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 17.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.5
    Not uninstalling wcwidth at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'wcwidth'. No files were found to uninstall.

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 5.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 4.5 MB/s eta 0:00

In [ ]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import html
import ftfy

from sklearn.svm import SVC
from xgboost import XGBClassifier
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# 2. Data Loading

In [ ]:
train = pd.read_csv("/work/dataset_penyisihan_bdc_2024.csv", sep=";")
test = pd.read_csv("/work/dataset_unlabeled_penyisihan_bdc_2024.csv", sep=";")

In [ ]:
train

,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik
...,...,...
4995,RT @l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwS...,Politik
4996,Masyarakat yakin bahwa Prabowo-Gibran memiliki...,Politik
4997,imo both are irrational but yg satu jauh lebih...,Ekonomi
4998,@cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM= ...,Pertahanan dan Keamanan


# 3. Data Preprocessing

In [ ]:
# Baseline data
train_baseline = train.copy()
print(train_baseline["text"])

0       Kunjungan Prabowo ini untuk meresmikan dan men...
1       RT Anies dapat tepuk tangan meriah saat jadi R...
2       @CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...
3       RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...
4       Anies Baswedan Harap ASN termasuk TNI dan Polr...
                              ...                        
4995    RT @l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwS...
4996    Masyarakat yakin bahwa Prabowo-Gibran memiliki...
4997    imo both are irrational but yg satu jauh lebih...
4998    @cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM= ...
4999    Acara ini tidak hanya memasak, tetapi Calon Pr...
Name: text, Length: 5000, dtype: object


In [ ]:
train_baseline["text"][2]

'@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= emng bener sih, pendukung 01 ada yg goblok, begitu jg dg pendukung 02.. hnya sj menurut pak Ridwan Kamil skemanya terbalik, klo 01 MAYORITAS PENDIDIKAN MENENGAH ATAS (artinya ada jg pendidikan rendah yg milih'

## 3.1 Text Cleaning

In [ ]:
# Remove Link, retweet, hashtag, emojis, and so on

def remove_link(x):
    result = re.sub(r"https?://\S+|www\.\S+", "", x)
    return result.strip()
def remove_retweet(x):
    # Dampak ke Kata Benda RT/RW
    result = re.sub(r"\bRT\b", "", x).strip()
    return result


def extract_metadata(x):
    match = re.search(r"\[(.*?)\]", x)
    return match.group(1) if match else None

def remove_metadata(x):
    return re.sub(r"\[(.*?)\]","", x).strip()
    
def extract_mentions(x):
    return re.findall(r"@[\S]+", x)

def remove_mentions(x):
    return re.sub(r'@[\S]+', "", x).strip()


def extract_hashtags(x):
    return re.findall(r"#\w+", x)
def remove_hashtags(x):
    return re.sub(r'#\w+', "", x).strip()

def remove_whatever_thisis(x):
    return re.sub(r"\B\/\w+\b", "", x).strip()


def transform_emojis(x):
    return ftfy.fix_encoding(x)

def remove_emojis():
    pass

train_baseline["metadata"] = train_baseline["text"].apply(extract_metadata)
train_baseline["cleaned"] = train_baseline["text"].apply(remove_metadata)

train_baseline["cleaned"] = train_baseline["cleaned"].apply(transform_emojis)

train_baseline["cleaned"] = train_baseline["cleaned"].apply(remove_link)
train_baseline["cleaned"] = train_baseline["cleaned"].apply(remove_retweet)    

train_baseline["raw_mentions"] = train_baseline["text"].apply(extract_mentions)
train_baseline["cleaned"] = train_baseline["cleaned"].apply(remove_mentions)



train_baseline["cleaned"] = train_baseline["cleaned"].apply(remove_whatever_thisis)



train_baseline['hashtag'] = train_baseline['text'].apply(extract_hashtags)
train_baseline["cleaned"] = train_baseline["cleaned"].apply(remove_hashtags)


#Test cleaning
test["metadata"] = test["Text"].apply(extract_metadata)
test["cleaned"] = test["Text"].apply(remove_metadata)

test["cleaned"] = test["cleaned"].apply(transform_emojis)

test["cleaned"] = test["cleaned"].apply(remove_link)
test["cleaned"] = test["cleaned"].apply(remove_retweet)    

test["raw_mentions"] = test["Text"].apply(extract_mentions)
test["cleaned"] = test["cleaned"].apply(remove_mentions)

test["cleaned"] = test["cleaned"].apply(remove_whatever_thisis)

test['hashtag'] = test['Text'].apply(extract_hashtags)
test["cleaned"] = test["cleaned"].apply(remove_hashtags)

In [ ]:
train_baseline["cleaned"]

0       Kunjungan Prabowo ini untuk meresmikan dan men...
1       Anies dapat tepuk tangan meriah saat jadi Rekt...
2       emng bener sih, pendukung 01 ada yg goblok, be...
3       Sewaktu anies bersikap kritis ke kinerja pak p...
4       Anies Baswedan Harap ASN termasuk TNI dan Polr...
                              ...                        
4995    Ngeliat debat kemaren pas prabowo kicep, kekir...
4996    Masyarakat yakin bahwa Prabowo-Gibran memiliki...
4997    imo both are irrational but yg satu jauh lebih...
4998    look at that pak ganjar, anda sdh berkecimpung...
4999    Acara ini tidak hanya memasak, tetapi Calon Pr...
Name: cleaned, Length: 5000, dtype: object

In [ ]:
# Make text lowercase, remove text in square brackets, remove punctuation, remove number
def clean_text(x):
    x = x.lower()
    x =  re.sub('\[.*?\]', '', x)
    return x

def transform_separator(x):
    return x.replace("-", " ")




def remove_punct(x):
    return x.translate(str.maketrans("","",string.punctuation)).strip()
train_baseline["cleaned"] = train_baseline["cleaned"].apply(transform_separator)
train_baseline["cleaned"] = train_baseline["cleaned"].apply(lambda x: clean_text(x))

train_baseline["cleaned"] = train_baseline["cleaned"].apply(remove_punct)

test["cleaned"] = test["cleaned"].apply(transform_separator)
test["cleaned"] = test["cleaned"].apply(lambda x: clean_text(x))

test["cleaned"] = test["cleaned"].apply(remove_punct)

## 3.3 Tokenize

In [ ]:

train_baseline["tokenize"] = train_baseline["cleaned"].apply(lambda x:nltk.tokenize.word_tokenize(x))
test["tokenize"] = test["cleaned"].apply(lambda x:nltk.tokenize.word_tokenize(x))


In [ ]:
def normalize_word(word):
    return re.sub(r'(.)\1{2,}', r'\1', word)

def normalize_words(words):
    return [normalize_word(word) for word in words]
test["tokenize"] = test["tokenize"].apply(lambda x: normalize_words(x))

train_baseline["tokenize"] = train_baseline["tokenize"].apply(lambda x: normalize_words(x))

## 3.4 Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
# # Time Consuming, 20 min+-ish
# def stem_verbose(x,i):
#     print("data[i]: x")
#     return [stemmer.stem(token) for token in x] 
train_baseline["stemmed"] = train_baseline["tokenize"].apply(lambda x: [stemmer.stem(token) for token in x] )
test["stemmed"] = test["tokenize"].apply(lambda x: [stemmer.stem(token) for token in x] )

In [ ]:
train_baseline

,text,label,metadata,cleaned,raw_mentions,hashtag,tokenize,stemmed
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,None,kunjungan prabowo ini untuk meresmikan dan men...,[],"[#IndonesiaSentris, #IndonesiaHijau, #02Melanj...","[kunjungan, prabowo, ini, untuk, meresmikan, d...","[kunjung, prabowo, ini, untuk, resmi, dan, ser..."
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,RE ekowboy2,anies dapat tepuk tangan meriah saat jadi rekt...,[],[#AminMiskinkanKoruptor],"[anies, dapat, tepuk, tangan, meriah, saat, ja...","[anies, dapat, tepuk, tangan, riah, saat, jadi..."
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,None,emng bener sih pendukung 01 ada yg goblok begi...,[@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y=],[],"[emng, bener, sih, pendukung, 01, ada, yg, gob...","[emng, bener, sih, dukung, 01, ada, yg, goblok..."
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,RE coldthem,sewaktu anies bersikap kritis ke kinerja pak p...,[@L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7Q=],[],"[sewaktu, anies, bersikap, kritis, ke, kinerja...","[waktu, anies, sikap, kritis, ke, kerja, pak, ..."
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,None,anies baswedan harap asn termasuk tni dan polr...,[],[],"[anies, baswedan, harap, asn, termasuk, tni, d...","[anies, baswedan, harap, asn, masuk, tni, dan,..."
...,...,...,...,...,...,...,...,...
4995,RT @l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwS...,Politik,RE BinNuh_,ngeliat debat kemaren pas prabowo kicep kekira...,[@l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwSo=],[],"[ngeliat, debat, kemaren, pas, prabowo, kicep,...","[ngeliat, debat, kemaren, pas, prabowo, kicep,..."
4996,Masyarakat yakin bahwa Prabowo-Gibran memiliki...,Politik,None,masyarakat yakin bahwa prabowo gibran memiliki...,[],[#KosongDuaSatuPutaran],"[masyarakat, yakin, bahwa, prabowo, gibran, me...","[masyarakat, yakin, bahwa, prabowo, gibran, mi..."
4997,imo both are irrational but yg satu jauh lebih...,Ekonomi,None,imo both are irrational but yg satu jauh lebih...,[],[],"[imo, both, are, irrational, but, yg, satu, ja...","[imo, both, are, irrational, but, yg, satu, ja..."
4998,@cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM= ...,Pertahanan dan Keamanan,None,look at that pak ganjar anda sdh berkecimpung ...,[@cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM=],[],"[look, at, that, pak, ganjar, anda, sdh, berke...","[look, at, that, pak, ganjar, anda, sdh, kecim..."


## 3.5 Remove Emoji & Translate

In [ ]:
# def remove_emoji(text):
#     emoji_pattern = re.compile("["
#     u"\U0001F600-\U0001F64F" #emoticons
#     u"\U0001F300-\U0001F5FF" #symbols & pictographs
#     u"\U0001F680-\U0001F6FF" #transport & map symbols
#     u"\U0001F1E0-\U0001F1FF" #flags (iOS)
#     u"\U00002702-\U000027B0"
#     u"\U000024C2-\U0001F251"
#     "]+", flags=re.UNICODE)
#     return emoji_pattern.sub(r'', text)
# train_baseline["tokenize"] =  train_baseline["tokenize"].apply(lambda x: remove_emoji(x)) 
import concurrent.futures
from googletrans import Translator
from tqdm import tqdm
tqdm.pandas()
from collections import defaultdict
def translate_word(word, translator, cache):
    if word in cache:
        return cache[word]
    try:
        translated_word = translator.translate(word, dest='id').text
        cache[word] = translated_word
        return translated_word
    except Exception as e:
        return word  # Return the original word if translation fails


def translate_words(words, translator, cache):
    word_with_index = [(index, word) for index, word in enumerate(words)]
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_word = {executor.submit(translate_word, word, translator, cache): (index, word) for index, word in word_with_index}
        translated_words = [None] * len(words)
        
        for future in concurrent.futures.as_completed(future_to_word):
            index, word = future_to_word[future]
            try:
                translated_words[index] = future.result()
            except Exception as exc:
                translated_words[index] = word  # Return the original word if translation fails
    return translated_words

cache = defaultdict(str)
translator = Translator()
train_baseline["stemmed"] = train_baseline["stemmed"].progress_apply(lambda x: translate_words(x, translator, cache))
test["stemmed"] = test["stemmed"].progress_apply(lambda x: translate_words(x, translator, cache))


100%|██████████| 1000/1000 [57:30<00:00,  3.45s/it]


In [ ]:
# Convert slang
import json

slang_words = json.load(open("combined_slang_words.txt"))
slang_words

{'@': 'di',
 'abis': 'habis',
 'ad': 'ada',
 'adlh': 'adalah',
 'afaik': 'as far as i know',
 'ahaha': 'haha',
 'aj': 'saja',
 'ajep-ajep': 'dunia gemerlap',
 'ak': 'saya',
 'akika': 'aku',
 'akkoh': 'aku',
 'akuwh': 'aku',
 'alay': 'norak',
 'alow': 'halo',
 'ambilin': 'ambilkan',
 'ancur': 'hancur',
 'anjrit': 'anjing',
 'anter': 'antar',
 'ap2': 'apa-apa',
 'apasih': 'apa sih',
 'apes': 'sial',
 'aps': 'apa',
 'aq': 'saya',
 'aquwh': 'aku',
 'asbun': 'asal bunyi',
 'aseekk': 'asyik',
 'asekk': 'asyik',
 'asem': 'asam',
 'aspal': 'asli tetapi palsu',
 'astul': 'asal tulis',
 'ato': 'atau',
 'au ah': 'tidak mau tahu',
 'awak': 'saya',
 'ay': 'sayang',
 'ayank': 'sayang',
 'b4': 'sebelum',
 'bakalan': 'akan',
 'bandes': 'bantuan desa',
 'bangedh': 'banget',
 'banpol': 'bantuan polisi',
 'banpur': 'bantuan tempur',
 'basbang': 'basi',
 'bcanda': 'bercanda',
 'bdg': 'bandung',
 'begajulan': 'nakal',
 'beliin': 'belikan',
 'bencong': 'banci',
 'bentar': 'sebentar',
 'ber3': 'bertiga',
 'b

In [ ]:
# train_baseline['final'] = train_baseline['stemmed'].apply(lambda x: [slang_words[word] if word in slang_words else word for word in x])
# test['final'] = test['stemmed'].apply(lambda x: [slang_words[word] if word in slang_words else word for word in x])

In [ ]:
def transform_word(word, slang_dict):
    if word in slang_dict:
        return slang_dict[word].split()
    else:
        return [word]

train_baseline['final'] = train_baseline['stemmed'].apply(
    lambda x: [sub_word for word in x for sub_word in transform_word(word, slang_words)]
)

test['final'] = test['stemmed'].apply(
    lambda x: [sub_word for word in x for sub_word in transform_word(word, slang_words)]
)

def clean_word(word):
    # Remove numbers and punctuation
    return re.sub(r'[0-9\W_]+', '', word).strip()

def remove_empty_and_whitespace(words):
    cleaned_words = [clean_word(word) for word in words]
    return [word for word in cleaned_words if word]


train_baseline['final'] = train_baseline['final'].apply(remove_empty_and_whitespace)

test['final'] = test['final'].apply(remove_empty_and_whitespace)

In [ ]:
display(train_baseline)

,text,label,metadata,cleaned,raw_mentions,hashtag,tokenize,stemmed,final
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,None,kunjungan prabowo ini untuk meresmikan dan men...,[],"[#IndonesiaSentris, #IndonesiaHijau, #02Melanj...","[kunjungan, prabowo, ini, untuk, meresmikan, d...","[kunjung, prabowo, ini, untuk, resmi, dan, ser...","[kunjung, prabowo, ini, untuk, resmi, dan, ser..."
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,RE ekowboy2,anies dapat tepuk tangan meriah saat jadi rekt...,[],[#AminMiskinkanKoruptor],"[anies, dapat, tepuk, tangan, meriah, saat, ja...","[anies, dapat, tepuk, tangan, riah, saat, jadi...","[anies, dapat, tepuk, tangan, riah, saat, jadi..."
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,None,emng bener sih pendukung 01 ada yg goblok begi...,[@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y=],[],"[emng, bener, sih, pendukung, 01, ada, yg, gob...","[emng, bener, sih, dukung, 01, ada, yg, goblok...","[memang, benar, sih, dukung, ada, yang, bodoh,..."
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,RE coldthem,sewaktu anies bersikap kritis ke kinerja pak p...,[@L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7Q=],[],"[sewaktu, anies, bersikap, kritis, ke, kinerja...","[waktu, anies, sikap, kritis, ke, kerja, pak, ...","[waktu, anies, sikap, kritis, ke, kerja, pak, ..."
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,None,anies baswedan harap asn termasuk tni dan polr...,[],[],"[anies, baswedan, harap, asn, termasuk, tni, d...","[anies, baswedan, harap, asn, masuk, tni, dan,...","[anies, baswedan, harap, asn, masuk, tni, dan,..."
...,...,...,...,...,...,...,...,...,...
4995,RT @l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwS...,Politik,RE BinNuh_,ngeliat debat kemaren pas prabowo kicep kekira...,[@l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwSo=],[],"[ngeliat, debat, kemaren, pas, prabowo, kicep,...","[ngeliat, debat, kemaren, pas, prabowo, kicep,...","[melihat, debat, kemarin, pas, prabowo, kicep,..."
4996,Masyarakat yakin bahwa Prabowo-Gibran memiliki...,Politik,None,masyarakat yakin bahwa prabowo gibran memiliki...,[],[#KosongDuaSatuPutaran],"[masyarakat, yakin, bahwa, prabowo, gibran, me...","[masyarakat, yakin, bahwa, prabowo, gibran, mi...","[masyarakat, yakin, bahwa, prabowo, gibran, mi..."
4997,imo both are irrational but yg satu jauh lebih...,Ekonomi,None,imo both are irrational but yg satu jauh lebih...,[],[],"[imo, both, are, irrational, but, yg, satu, ja...","[imo, both, are, irrational, but, yg, satu, ja...","[imo, both, are, irrational, but, yang, satu, ..."
4998,@cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM= ...,Pertahanan dan Keamanan,None,look at that pak ganjar anda sdh berkecimpung ...,[@cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM=],[],"[look, at, that, pak, ganjar, anda, sdh, berke...","[look, at, that, pak, ganjar, anda, sdh, kecim...","[look, at, that, pak, ganjar, anda, sdh, kecim..."


## 3.6 Stopword 

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
list_stopword =  set(stopwords.words('indonesian'))

factory = StopWordRemoverFactory()
stop_words = factory.get_stop_words()

list_stopword = list(set(list_stopword).union(set(stop_words)))

def remove_stopword(text, set_stop_word):
    text = [word for word in text if word not in set_stop_word]
    return text

train_baseline["final"] = train_baseline["final"].apply(lambda x: remove_stopword(x, list_stopword))
test["final"] = test["final"].apply(lambda x: remove_stopword(x, list_stopword))


In [ ]:
# train_baseline = pd.read_excel(open('stemmed_view.xlsx', 'rb'),sheet_name='Sheet_name_1') 
with pd.ExcelWriter('final_view.xlsx') as writer:  
    train_baseline.to_excel(writer, sheet_name='Sheet1')

with pd.ExcelWriter('fina_testl_view.xlsx') as writer:  
    test.to_excel(writer, sheet_name='Sheet1')

In [ ]:
train_baseline.loc[train_baseline['cleaned'].isna()==True]

,text,label,metadata,cleaned,raw_mentions,hashtag,tokenize,stemmed,final


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=345f8878-fb72-4223-91e4-c32f6cff64f7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>